In [1]:
import sys
import time
import warnings
import logging
import config
import math
import torch
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from transformers import AutoModel
from torch import nn
from torch.nn import functional as F
from sklearn.metrics import (accuracy_score, roc_curve, auc)
warnings.filterwarnings("ignore")
logging.basicConfig(level=logging.ERROR)

I0729 18:49:43.043088 17656 file_utils.py:39] PyTorch version 1.2.0 available.
I0729 18:49:52.989889 17656 configuration_utils.py:285] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-uncased-config.json from cache at C:\Users\YangWang/.cache\torch\transformers\a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.8949e27aafafa845a18d98a0e3a88bc2d248bbc32a1b75947366664658f23b1c
I0729 18:49:52.990919 17656 configuration_utils.py:321] Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "vocab_size": 30522
}

I0729 18:49:54.424886 17656 token

In [2]:
from deep_portfolio import ReutersClassifier

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = ReutersClassifier(n_classes=2, top_k=10, p=0.1, window_size=3, out_channels=64)
model.load_state_dict(torch.load("../weights/2020-07-29_cnn_distilbert.bin"))
model.to(device)

I0729 18:49:57.342915 17656 configuration_utils.py:285] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-uncased-config.json from cache at C:\Users\YangWang/.cache\torch\transformers\a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.8949e27aafafa845a18d98a0e3a88bc2d248bbc32a1b75947366664658f23b1c
I0729 18:49:57.343915 17656 configuration_utils.py:321] Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "vocab_size": 30522
}

I0729 18:49:58.721887 17656 modeling_utils.py:650] loading weights file https://cdn.huggingface.co/distilbert-ba

ReutersClassifier(
  (distilbert_layer): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1):

In [3]:
valid_data = joblib.load("../data/valid_top10_v2.bin")
valid_data.index.min(), valid_data.index.max()

(datetime.date(2016, 1, 4), datetime.date(2016, 12, 30))

In [4]:
test_data = joblib.load("../data/test_top10_v2.bin")
print(test_data.index.min(), test_data.index.max())
test_data.head()

2017-01-03 2020-07-01


,Top 1 News,Top 2 News,Top 3 News,Top 4 News,Top 5 News,Top 6 News,Top 7 News,Top 8 News,Top 9 News,Top 10 News,Open,High,Low,Close,Volume,Dividends,Stock Splits,ticker,label
date,,,,,,,,,,,,,,,,,,,
2017-01-04,apple confirms $1 bln investment in softbank t...,update 2-apple confirms $1 bln investment in s...,apple confirms $1 billion investment in softba...,apple pulls new york times app from itunes sto...,NaN,NaN,NaN,NaN,NaN,NaN,110.11,110.73,110.01,110.27,21118100.0,0.0,0.0,AAPL,0
2017-01-05,brief-apple says app store generated over $20 ...,update 1-apple is app store generated $20 bln ...,apple is app store generated $20 billion for d...,rpt-update 3-apple pulls new york times apps i...,update 3-apple pulls new york times apps in ch...,apple pulls new york times apps in china after...,india reluctant to give special tax incentives...,"apple plans first retail store in s.korea, pos...",NaN,NaN,110.17,111.07,110.07,110.83,22193600.0,0.0,0.0,AAPL,1
2017-01-06,brief-apple inc is ceo tim cook is total 2016 ...,canada is competition watchdog closes two-year...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,110.99,112.30,110.70,112.07,31751900.0,0.0,0.0,AAPL,1
2017-01-10,china is wechat seeks slice of apple is app st...,china is wechat seeks slice of apple is app st...,tesla taps apple engineer for autopilot software,NaN,NaN,NaN,NaN,NaN,NaN,NaN,112.88,113.46,112.44,113.21,24462100.0,0.0,0.0,AAPL,1
2017-01-12,u.s. appeals court revives antitrust lawsuit a...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,113.01,113.39,112.35,113.34,27086200.0,0.0,0.0,AAPL,0


In [5]:
from collections import Counter
len(dict(Counter(test_data.ticker)))

99

In [6]:
from deep_portfolio import progressbar
from deep_portfolio import create_dataloader
from transformers import AutoTokenizer


PRE_TRAINED_MODEL_NAME = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)
test_dataloader = create_dataloader(test_data, tokenizer, 32, 10, 16, shuffle=False)
model = model.eval()
pred_list = []
with torch.no_grad():
    for data in progressbar(test_dataloader):
        for d in data["ids_and_mask"]:
            d["input_ids"] = d["input_ids"].to(device)
            d["attention_mask"] = d["attention_mask"].to(device)

        outputs = model(data["ids_and_mask"])
        outputs = F.softmax(outputs)
        pred_list.extend(outputs)

I0729 18:50:46.039817 17656 configuration_utils.py:285] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-uncased-config.json from cache at C:\Users\YangWang/.cache\torch\transformers\a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.8949e27aafafa845a18d98a0e3a88bc2d248bbc32a1b75947366664658f23b1c
I0729 18:50:46.042818 17656 configuration_utils.py:321] Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "vocab_size": 30522
}

I0729 18:50:48.697847 17656 tokenization_utils.py:1015] loading file https://s3.amazonaws.com/models.huggingface

[############################################################] 100%


In [31]:
pred_list_cpu = [pred.to("cpu").tolist() for pred in pred_list]
pred_list_final = [pred[1] for pred in pred_list_cpu]
pred_list_final = [pred for pred in pred_list_final]
test_data["pred"] = pred_list_final
final_df = test_data[["ticker", "pred"]]
final_df.reset_index(drop=False, inplace=True)
final_df["date"] = final_df["date"].apply(str)

In [32]:
df_total = pd.DataFrame()
for ticker in list(set(final_df.ticker)):
    index = pd.date_range(start="2017-01-01", end="2020-06-30", freq='D')
    columns = ["ticker", "pred"]
    df = pd.DataFrame(index=index, columns=columns)
    df = df.reset_index(drop=False)
    df.columns = ["date", "ticker", "pred"]
    df["date"] = df["date"].apply(str)
    
    df_temp = pd.merge(df, final_df[final_df["ticker"] == ticker], on="date")
    df_total = pd.concat([df_total, df_temp], axis=0)

In [33]:
df_total

,date,ticker_x,pred_x,ticker_y,pred_y


In [34]:
final_df.groupby("ticker").agg

,date,ticker,pred
0,2017-01-04,AAPL,0.529238
1,2017-01-05,AAPL,0.529238
2,2017-01-06,AAPL,0.529238
3,2017-01-10,AAPL,0.529238
4,2017-01-12,AAPL,0.529238
...,...,...,...
18937,2020-03-17,NOW,0.529238
18938,2020-04-21,NOW,0.529238
18939,2020-04-30,NOW,0.529238
18940,2020-05-05,NOW,0.529238


In [35]:
1277*99 - 18942

107481

In [20]:
18942/99

191.33333333333334